In [1]:
import re
import shutil
import time
from pathlib import Path

import mph
import numpy as np
import yaml

from sdisk import *

This is the main `jupyter notebook` to automate running eigenfrequency simulations in COMSOL.

First, we define all the relative paths to access the parameters, layouts, and results.

For this example, we will consider the the "sdisk" design, so we specify the directory containing layouts to `layouts_path = Path(r"layouts\sdisk\layouts")`

In [2]:
param_path = Path(r"layouts\sdisk\params")
layouts_path = Path(r"layouts\sdisk\layouts")
results_path = Path(r"results")

In [3]:
def load_results(index: int) -> np.ndarray: 
    name = Path(r"sdisk{}.csv".format(index))
    return np.genfromtxt(results_path/name)

In [4]:
def generate_yaml(param_path:Path, params:dict) -> None:
    i = 0
    for gds in param_path.glob("*.yml"):
        existing_i = int(re.findall(r'[0-9]+', gds.name)[0])
        if existing_i >= i:
            i = existing_i + 1
            
    yml_path = param_path/Path(r"sdisk{}.yml".format(i))
    
    with open(yml_path, "w") as out:
        yaml.dump(params, out, default_flow_style=False)

In [5]:
def read_yaml(param_path:Path, filename:Path, *variable:str):
    with open(param_path/filename, "r") as f:
        p = yaml.safe_load(f)
    
    if len(variable) == 1: 
        return p[variable[0]]
    else:
        return tuple([p[v] for v in variable]) 

The idea is that this notebook will 
1. Read a list of yml files 
2. From the yml files, generate the corresponding gds file 
3. COMSOL will perform the simulation using that gds file 
4. Save results to a specified directory

### GDS generation

For every new gds generated, the `generate_` functions automatically indexes the new files with some integer. 

In [ ]:
# arm_pitch_ratio depends on the thickness of the meander lines 
# thickness depends on the disk radius 
default_params = {'arm_inner_frac': 0.05,
                  'arm_n': 4,
                  'arm_outer_frac': 0.05,
                  'arm_pitch_ratio': 1.2,
                  'arm_thetas': [0, 90, 180, 270],
                  'arm_thick_frac': 0.05,
                  'arm_w_frac': 0.5,
                  'actual_disk_r': 500,
                  'disk_r': 500,
                  'inner_arc_frac': 0.25,
                  'ped_arc_frac': 0.1
                  }
    
for k in range(1, 10):
    default_params['arm_n'] = k
    generate_yaml(param_path, default_params)
    generate_gds(default_params)

### Loading and running the model

For more information on how to extend the capabilties of the this code, refer to the [documentation](https://mph.readthedocs.io/en/stable/) for the `mph` package.

The code automatically runs the eigenfrequency simulations by loading the comsol file `disk.mph` with a particular KLayout file. To change how many eigenfrequencies will be solve for requires manually changing this setting in the COMSOL GUI. After the simulation completes, the following are saves to the `results` directory: 
1. Labled and unlabeled images for the mode shape
2. Eigenfrequencies found 
3. Copy of the gds file 
4. Copy of the settings used to produce the gds file.  

In [6]:
# we can specify how many cores we want to use here by using the `core` keyword
client = mph.start()
# make sure your COMSOL file in the correct directory!
model = client.load('disk.mph')

In [7]:
for i in range(35, 36):
    gds = Path(r"sdisk{}.gds".format(i))
    param = Path(r"sdisk{}.yml".format(i))
    gds_name = gds.name.replace(".gds", "")

    gds_path = layouts_path/gds
    full_param_path = param_path/param
    
    ped_arc_frac, disk_r = read_yaml(param_path, Path(r"sdisk{}.yml".format(i)), "ped_arc_frac", "disk_r")
    pedestal_r = ped_arc_frac*disk_r
    print("Changing pedestal_r to {}...".format(pedestal_r))
    model.parameter("pedestal_r", f"{pedestal_r}[um]")
    
    print("Changing design to {}...".format(gds_name))
    model.property("geometries/Geometry 1/Import 1", "filename", str(gds_path))
    model.build()

    print("\tSolving... \U0001F600")
    t0 = time.time()
    model.solve(model.studies()[0])
    t1 = time.time()

    print("\tCompleted in {:.1f} seconds. Exporting...".format(t1-t0))

    table_name = gds_name + ".csv"
    table_path = results_path/Path(table_name)
    ef = model.evaluate("freq")
    np.savetxt(table_path, ef, header="Eigenfrequency (Hz)",)

    shutil.copy(gds_path, results_path/gds)
    shutil.copy(full_param_path, results_path/param)

    fig_dir = Path(results_path/Path(gds_name + "_labeled"))
    fig_dir.mkdir(parents=True, exist_ok=True)
    fig_path = fig_dir/Path(gds_name + "_")

    model.property("exports/Labeled", "imagefilename", str(fig_path))
    model.export('Labeled')

    fig_dir = Path(results_path/Path(gds_name + "_unlabeled"))
    fig_dir.mkdir(parents=True, exist_ok=True)
    fig_path = fig_dir/Path(gds_name + "_")

    model.property("exports/Unlabeled", "imagefilename", str(fig_path))
    model.export('Unlabeled')
    print("\tExport complete...")


Changing pedestal_r to 50.0...
Changing design to sdisk35...
	Solving... 😀
	Completed in 81.9 seconds. Exporting...
	Export complete...


In [8]:
# discard all the data and everything that we don't need
model.clear()
model.reset()
model.save()